### Splitting and Embedding Text Using LangChain

In [2]:
#!pip install pypdf

In [3]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('churchill-speech.txt') as f:
    churchill_speech = f.read()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [5]:
chunks = text_splitter.create_documents([churchill_speech])
# print(chunks[2])
# print(chunks[10].page_content)
print(f'Now you have {len(chunks)}')

Now you have 109


#### Embedding Cost

In [6]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')
    
print_embedding_cost(chunks)

Total Tokens: 2064
Embedding Cost in USD: 0.000826


### Creating embeddings

In [7]:
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

C:\Users\ajfon\Documents\virtuals\llm\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
vector = embeddings.embed_query(chunks[0].page_content)
#vector

### Inserting the Embeddings into a Pinecone Index

In [10]:
import os
import pinecone
from langchain.vectorstores import Pinecone

pinecone.init(api_key=os.environ.get('PINECONE_API_KEY'), environment=os.environ.get('PINECONE_ENV'))

C:\Users\ajfon\Documents\virtuals\llm\Lib\site-packages\pinecone\index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
# deleting all indexes
indexes = pinecone.list_indexes()
for i in indexes:
    print('Deleting all indexes ... ', end='')
    pinecone.delete_index(i)
    print('Done')

Deleting all indexes ... Done


In [12]:
# creating an index
index_name = 'churchill-speech'
if index_name not in pinecone.list_indexes():
    print(f'Creating index {index_name} ...')
    pinecone.create_index(index_name, dimension=1536, metric='cosine')
    print('Done!')

Creating index churchill-speech ...
Done!


In [13]:
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

### Asking Questions (Similarity Search)

In [14]:
query = 'Where should we fight?'
result = vector_store.similarity_search(query)
print(result)

[Document(page_content='In a long series of very fierce battles, now on this front, now on that, fighting on three fronts'), Document(page_content='in the air. We shall defend our island whatever the cost may be; we shall fight on beaches, landing'), Document(page_content='a defensive war. We have our duty to our Allies.'), Document(page_content='equal or sometimes larger number of the enemy, and fought very fiercely on old ground so many of us')]


In [15]:
for r in result:
    print(r.page_content)
    print('-' * 50)

In a long series of very fierce battles, now on this front, now on that, fighting on three fronts
--------------------------------------------------
in the air. We shall defend our island whatever the cost may be; we shall fight on beaches, landing
--------------------------------------------------
a defensive war. We have our duty to our Allies.
--------------------------------------------------
equal or sometimes larger number of the enemy, and fought very fiercely on old ground so many of us
--------------------------------------------------


### Answering in Natural Language using an LLM

In [16]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)

retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 3})

chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)


C:\Users\ajfon\Documents\virtuals\llm\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [17]:
query = 'Where should we fight?'
answer = chain.run(query)
print(answer)

C:\Users\ajfon\Documents\virtuals\llm\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


We should fight on our island, on the beaches, and in the air.


In [19]:
query = 'Who was the king of Belgium at that time?'
query = 'What about the French Armies??'
answer = chain.run(query)
print(answer)

The French Army has been weakened and suffered significant losses.
